# Протоколы обмена ключами

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

In [3]:
def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

In [4]:
def randint(a, b):
    """
    Генерация слуайного числа среди заданных.
    """
    
    native_a = int(a)
    native_b = int(b)
    return gmpy2.mpz(random.randint(native_a, native_b))

----

In [5]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime
from Crypto.Random import random

# HKDF и то, что надо для нее
from Crypto.Protocol.KDF import HKDF
from Crypto.Hash import SHA512

# Подпись и шифрование
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Cipher import AES

# Математическая библиотека
import gmpy2

# Для генерации идентификаторов
import uuid

# Утилитки
from Crypto.Util.number import long_to_bytes
import binascii

----

## Протокол Диффи-Хеллмана
![Diffe-Hellman Protocol](./images/Diffie-Hellman_4.png)

In [6]:
class DiffieHellmanParams(object):
    def __init__(self, prime_bits: int = 512):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[DiffieHellmanParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [7]:
class DiffieHellmanUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: DiffieHellmanParams, 
                 use_another_value: bool = False):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        self._sabotage = use_another_value
        
        trace('[DiffieHellmanUser]', f'client {self._id} created')
    
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, отправляю другому пользователю
        # Получаю от него g ** y mod p
        #
        
        x = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{x = }')
        
        intermediate_value = another_client._accept_key_exchange(gmpy2.powmod(self._g, x, self._p))
        
        #
        # Если требуют неудачную попытку, то подменю x
        # Это как бы незнание изначального значения - попытка угадать
        #
        
        if self._sabotage:
            x = self._generate_group_element()
            trace('[DiffieHellmanUser]', f'[{self._id}]', f'guessed {x = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value) -> int:
        #
        # Получаю g ** x mod p, генерирую у
        #
        
        y = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{y = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return gmpy2.powmod(self._g, y, self._p)
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [8]:
# Параметры протокола
params = DiffieHellmanParams()

[TRACE] [DiffieHellmanParams] p = 12084714093411605017891085314500492391779373972043377589149908554934882367705599394182188633577232667612424554301905540934507123231316491188745535938455621, g = 9967553925170674168626172345506684212980532990584631497251252822123950168810113539594513357359262280530111931475246469113987548689983729155885027869031436


In [9]:
# Два хороших пользователя
alice = DiffieHellmanUser(params)
bob   = DiffieHellmanUser(params)

[TRACE] [DiffieHellmanUser] client 1995782b-a1fb-4601-9169-36edacb2fcee created
[TRACE] [DiffieHellmanUser] client 61775d00-2379-4715-8e2d-1ec654e174aa created


In [10]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [1995782b-a1fb-4601-9169-36edacb2fcee] x = mpz(2404433793977967968639298828223993352383498527526758447667401705624348381235327887971807015624714512370527397712718427142599948265735783416008212086670436)
[TRACE] [DiffieHellmanUser] [61775d00-2379-4715-8e2d-1ec654e174aa] y = mpz(8655556869741626848472569534989046501290198892690098439820813767197632823112689300710524922996017998304952338836840412227632497019961469744795978336244745)
[TRACE] [DiffieHellmanUser] [61775d00-2379-4715-8e2d-1ec654e174aa] derived key: f424d46fa82a8f6819311b0a626bf2e367c18a108178f9070204c0335725fc02
[TRACE] [DiffieHellmanUser] [1995782b-a1fb-4601-9169-36edacb2fcee] derived key: f424d46fa82a8f6819311b0a626bf2e367c18a108178f9070204c0335725fc02


In [11]:
# Теперь врага создам
mallory = DiffieHellmanUser(params, True)

[TRACE] [DiffieHellmanUser] client 76cf0793-5585-4359-97ad-8a7ce30f77af created


In [12]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [76cf0793-5585-4359-97ad-8a7ce30f77af] x = mpz(11380551920749555116401420131855013519742062354481121021235322606682385043013587745102119454849096051264029281044435249750511445378703139486989338360728120)
[TRACE] [DiffieHellmanUser] [61775d00-2379-4715-8e2d-1ec654e174aa] y = mpz(6110232113925736023263011748167728105427608339015640867934129807018531982604816265237804370126652259954809680360135082954694247862208404959394520529946594)
[TRACE] [DiffieHellmanUser] [61775d00-2379-4715-8e2d-1ec654e174aa] derived key: 7055231763b6001cf729197416a3e37b791d3bf81e124af16a7f0ecbf82ffe43
[TRACE] [DiffieHellmanUser] [76cf0793-5585-4359-97ad-8a7ce30f77af] guessed x = mpz(7797158925604243922627774559874878911437329411784201208262544675763956992378741249739413477034547141307748013773938796776065416755308374659569078809256735)
[TRACE] [DiffieHellmanUser] [76cf0793-5585-4359-97ad-8a7ce30f77af] derived key: e9f38e3db9e9d89385df5377ba003ed94b2cd9a0d9fac14fab4489dbd9f5a00b


----
## Протокол MTI
![Diffe-Hellman Protocol](./images/MTI_4.png)

In [13]:
class MTIParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[MTIParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [14]:
class MTIUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: MTIParams, *,
                 fake_data: gmpy2.mpz = None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = self._generate_group_element()
        self._public_key  = gmpy2.powmod(self._g, self._private_key, self._p) if fake_data is None else fake_data
        
        trace('[MTIUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
        
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому клиенту и получаю от него g ** y mod p
        #
        
        intermediate_value = another_client._accept_key_exchange(m, self.public_key)
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(another_client.public_key, x, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
    
    
    def _accept_key_exchange(self, intermediate_value, public_key):
        #
        # Генерирую у, вычисляю g ** y mod p
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(public_key, y, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(MTIUser.HKDF_DUMMY_STUFF, 
                         MTIUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return m
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [15]:
# Параметры протокола
params = MTIParams(512)

[TRACE] [MTIParams] p = 9888926915479633119867110849228648353728849415766763527377413427841437882867887479780377435649995010913913220833927137453510758446431231077352314322032357, g = 3988709560929595758222026731766588010717970304598577205070172239047115739881507133847351380930748226975347713065616398788784918988897480048320751584601905


In [16]:
# Два хороших пользователя
alice = MTIUser(params)
bob   = MTIUser(params)

[TRACE] [MTIUser] client aa7597c5-5b00-4d0a-95f4-dcf3f6c203b9 created. PK = 3976243981478272960374982406828104139514106886116060448454522575265898133518795434275167311778822451456608141930977180188231545273204539620318116295562286, SK = 4886021660988512551952591073533949523706417351724705917827161083127275861547456603740575965932657945186477609643326849621873629893527092058935392667317988
[TRACE] [MTIUser] client fb840f2b-1748-44b0-ba4c-77a7d1c5f20e created. PK = 4901140387453078600435245062425731978483552154410570243958083930848124921218972122816546412270016608288009679757204064957820687412447821691835246939535436, SK = 2040782717457316005280647830342260979516386509749708323923074173116884408513937016901473040198153303037782033434484433056146332473473921702501957609906280


In [17]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [MTIUser] [aa7597c5-5b00-4d0a-95f4-dcf3f6c203b9] x = mpz(93514748394166819965224678686389827591436512231196707708618288436044202034798542314250345851972798991036424483345245565545583771399768862693664260682197), m = mpz(1362934752220779663499844019094853081429794881871219117053252305533395229507043036297116375778749927557435488819665235606656951290062695563387191512361191)
[TRACE] [MTIUser] [fb840f2b-1748-44b0-ba4c-77a7d1c5f20e] y = mpz(981156213626691415794729448832337218368447508966576076104883475992534908681570309340240590839051681141955641147166446912697242223469544719899948990773371), m = mpz(277448465162834622465147589285158627587218833922071615285441723152418198347579232204864438214405425805522972484579542194922181064178261408133758192940527)
[TRACE] [MTIUser] [fb840f2b-1748-44b0-ba4c-77a7d1c5f20e] derived key: 2dc47e5508338ad9c488669fdcd4345dd37a548282f1555d356789b3d3c3891c
[TRACE] [MTIUser] [aa7597c5-5b00-4d0a-95f4-dcf3f6c203b9] derived key: 2dc47e5508338ad9c488669fdcd

In [18]:
# Теперь врага создам
mallory = MTIUser(params, fake_data=alice.public_key)

[TRACE] [MTIUser] client 00e18358-4205-4e75-8931-c4fdf2e3ec47 created. PK = 3976243981478272960374982406828104139514106886116060448454522575265898133518795434275167311778822451456608141930977180188231545273204539620318116295562286, SK = 9129385536472394000033574868911350039916516465806252848603123064186427600982639685609356684006810618839198703844016092643746389920975036727949430240834906


In [19]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [MTIUser] [00e18358-4205-4e75-8931-c4fdf2e3ec47] x = mpz(4849310966694300152072172452314821446540587537951593216127482562433446065236221271848596385969425272240428592043970479066348729255975018103535055646339988), m = mpz(4664296895039636798838387936579970868811283090050235960815954547765083641490360569082809317015234227216753832502618568645322421742653998474819736602470298)
[TRACE] [MTIUser] [fb840f2b-1748-44b0-ba4c-77a7d1c5f20e] y = mpz(6750090901278771775906628050918171275751229176143570881207696684737918704705019426923280088925903834173216559800683148978463731083224839183959190250117447), m = mpz(6563690575874369536622211607777676732123853019650604046018795783475150694058067153817489740313294464351711652161817854673126832902636781465203820004824226)
[TRACE] [MTIUser] [fb840f2b-1748-44b0-ba4c-77a7d1c5f20e] derived key: dcb5eb9ad7e18d2df6915a0733432cbda1ca8a5260310e745e554312409fc4f4
[TRACE] [MTIUser] [00e18358-4205-4e75-8931-c4fdf2e3ec47] derived key: 96daad2702ce075450ebbf5

----
## Протокол STS
![STS Protocol](./images/STS_4.png)

In [20]:
class STSParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[STSParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [21]:
class STSUser(object):
    RSA_KEY_LENGTH = 4096
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: STSParams, *, 
                 fake_data=None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = RSA.generate(STSUser.RSA_KEY_LENGTH)
        self._public_key  = self._private_key.publickey() if fake_data is None else fake_data
        
        trace('[STSUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому пользователю, получаю g ** y mod p и зашифрованную подпись
        #
        
        intermediate_value, encrypted_signature = another_client._accept_key_exchange(m)
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Расшифровываю подпись и проверяю ее
        # Если не подошла, то ключ неверный, завершаюсь
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(intermediate_value, m, received_signature, another_client.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return
        
        #
        # Сам теперь подписываю значения и отправляю на проверку другому пользователю
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        if not another_client._check_key_exchange(m, intermediate_value, 
                                                  (cipher.nonce, cipher.encrypt(signature)), 
                                                  self.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Another user refused key exchange')
            self._key = None
            return
        
        #
        # Тут все хорошо
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value):
        #
        # Генерирую у, вычисляю g ** y mod p
        # Вывожу ключ
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Создаю подпись значений и отправляю инициатору
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        encrypted_signature = (cipher.nonce, cipher.encrypt(signature))
        
        return m, encrypted_signature
    
    
    def _check_key_exchange(self, f, s, encrypted_signature, pk):
        #
        # Проверяю подпись от инициатора
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(f, s, received_signature, pk):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return False
        
        #
        # Если все валидно, то ключ верно установлен
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        return True
        
    
    def _sign_values(self, f, s):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        return pss.new(self._private_key).sign(h)
    
    
    def _check_signature(self, f, s, signature, pk):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        verifier = pss.new(pk)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
    
        
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key

-----

In [22]:
# Параметры протокола
params = STSParams(512)

[TRACE] [STSParams] p = 11368317696381622035834900956574139901281695373406661476722504794136967761080150137587316206655883293847550371873219678211843784550627998533823779586812471, g = 7816131149596441793038842546557204665947555653112675990926729818076487153235813175321200951235457988016746207246967307066893978135962904191025851720710207


In [23]:
# Два хороших пользователя
alice = STSUser(params)
bob   = STSUser(params)

[TRACE] [STSUser] client f72b1fad-3b28-4ce4-b67a-79ef0786ab57 created. PK = Public RSA key at 0x1072676D0, SK = Private RSA key at 0x1072652E0
[TRACE] [STSUser] client c30f842b-1842-4721-a144-f879a1a7f864 created. PK = Public RSA key at 0x1072655E0, SK = Private RSA key at 0x1072839D0


In [24]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [STSUser] [f72b1fad-3b28-4ce4-b67a-79ef0786ab57] x = mpz(1154837496969553956569984768834947440183969398573386846893811974271519198502474059571491299063380873063465625077723595979491967997548094505133087031966313), m = mpz(7006996024979780910519933724231988913877724260480340680166703696779764879284966592118859026941997628991062994534097122801466573731909382426249052876653101)
[TRACE] [STSUser] [c30f842b-1842-4721-a144-f879a1a7f864] y = mpz(1372354035655722011711163195854018306460801169919841914107154228384216903657287557095719697528427679415272993232998326758615678115205359143065875318317483), m = mpz(2226336538477365028620829812894378344951243319422941180953043886692596082569040644017181346950161916183749409921039171313493064931062184780211347435967838)
[TRACE] [STSUser] [c30f842b-1842-4721-a144-f879a1a7f864] derived key: e79406137f6d2d7dd659d00ccd4dfae2c52b1bb4c6737372f59463d913ec27dd
[TRACE] [STSUser] [f72b1fad-3b28-4ce4-b67a-79ef0786ab57] derived key: e79406137f6d2d7dd659d00

In [25]:
# Теперь врага создам
mallory = STSUser(params, fake_data=alice.public_key)

[TRACE] [STSUser] client aeaabe1e-2583-4860-96d4-ce68104d4ebd created. PK = Public RSA key at 0x1072676D0, SK = Private RSA key at 0x1072924F0


In [26]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [STSUser] [aeaabe1e-2583-4860-96d4-ce68104d4ebd] x = mpz(11082433273154170785710364361794641471068200908435816603474788571411603165115902783036713130401706866937292453875718006684032482874902216554358292693942624), m = mpz(3102278776262993289360852361183985824890853079929639935496029940356650426915307195806868561394399047017657420091956154482047317716745345171497757173026837)
[TRACE] [STSUser] [c30f842b-1842-4721-a144-f879a1a7f864] y = mpz(5694377477394150379029051427358631853364726952536203338758005061798639960137493332025672647545115200264741310952479055236355651332022039050863039983849975), m = mpz(9666907080696096804921974697918526847270955959578377408006717673259383660613152770663077872069509174715777007055827223117771268549269389409546932814973729)
[TRACE] [STSUser] [c30f842b-1842-4721-a144-f879a1a7f864] Invalid signature
[TRACE] [STSUser] [aeaabe1e-2583-4860-96d4-ce68104d4ebd] Another user refused key exchange


In [27]:
bob.initiate_key_exchange(mallory)

[TRACE] [STSUser] [c30f842b-1842-4721-a144-f879a1a7f864] x = mpz(787228239747013430808474611575481832539197975414002218032441809422166315289202589463270356082041776881423720415175712397826688415802721296390158714494862), m = mpz(10294885108791021230150292672015754233541289801534695066326998053093547384919487808501882199250831857948484346633263853239691783609643251042947389425895831)
[TRACE] [STSUser] [aeaabe1e-2583-4860-96d4-ce68104d4ebd] y = mpz(1270158543230820095477019677790500904407713950689217412067163517832502087887351383883401455864259619545410449017004184958353368812886593450873403884617512), m = mpz(3159158310224617998224272389442834835763268212897231387603212382024143048795421023470745810155027348558842054710756919711455132186766246702905057662417790)
[TRACE] [STSUser] [c30f842b-1842-4721-a144-f879a1a7f864] Invalid signature
